In [2]:
%load_ext nb_black
import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
from datetime import datetime
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import utils

from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator
import jupyter_notebook.stats_t as stats
import matplotlib.pyplot as plt
import warnings
import sklearn.metrics as metrics

# from ipython_utils import cal_acc, show_results
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats import percentileofscore
from sklearn.metrics import roc_auc_score, accuracy_score

warnings.filterwarnings("ignore")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [3]:
disease = "effusion"
path = f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/{disease}"
test_attr = torch.load(os.path.join(path, "dataset_g", "test_attributes.pt"))
test_proba_concept = torch.load(
    os.path.join(path, "dataset_g", "test_proba_concepts.pt")
)
test_logits_concept = torch.load(
    os.path.join(path, "dataset_g", "test_logits_concepts.pt")
)
test_logits_bb = torch.load(os.path.join(path, "dataset_g", "test_logits_bb.pt"))
test_class_labels = torch.load(os.path.join(path, "dataset_g", "test_class_labels.pt"))

train_attr = torch.load(os.path.join(path, "dataset_g", "train_attributes.pt"))
train_proba_concept = torch.load(
    os.path.join(path, "dataset_g", "train_proba_concepts.pt")
)
train_logits_concept = torch.load(
    os.path.join(path, "dataset_g", "train_logits_concepts.pt")
)
train_logits_bb = torch.load(os.path.join(path, "dataset_g", "train_logits_bb.pt"))
train_class_labels = torch.load(
    os.path.join(path, "dataset_g", "train_class_labels.pt")
)

val_attr = torch.load(os.path.join(path, "dataset_g", "val_attributes.pt"))
val_proba_concept = torch.load(os.path.join(path, "dataset_g", "val_proba_concepts.pt"))
val_logits_concept = torch.load(
    os.path.join(path, "dataset_g", "val_logits_concepts.pt")
)
val_logits_bb = torch.load(os.path.join(path, "dataset_g", "val_logits_bb.pt"))
val_class_labels = torch.load(os.path.join(path, "dataset_g", "val_class_labels.pt"))

print(
    test_attr.size(),
    test_proba_concept.size(),
    test_logits_concept.size(),
    test_logits_bb.size(),
    test_class_labels.size(),
)
print()
print(
    train_attr.size(),
    train_proba_concept.size(),
    train_logits_concept.size(),
    train_logits_bb.size(),
    train_class_labels.size(),
)
print()
print(
    val_attr.size(),
    val_proba_concept.size(),
    val_logits_concept.size(),
    val_logits_bb.size(),
    val_class_labels.size(),
)

with open(os.path.join(path, "MIMIC_test_configs.pkl"), "rb") as input_file:
    args = pickle.load(input_file)

concepts = args.landmark_names_spec + args.abnorm_obs_concepts
print(len(concepts))

torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([48144]) torch.Size([24072, 1])

torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([375096]) torch.Size([187548, 1])

torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([46898]) torch.Size([23449, 1])
107


<IPython.core.display.Javascript object>

In [4]:
disease_index = concepts.index(disease)
concepts.index(disease), concepts[concepts.index(disease)]

(46, 'effusion')

<IPython.core.display.Javascript object>

In [17]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
all_auroc = []
sub_auroc = []
sub_recall = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    all_auroc.append(auroc_val)
    if auroc_val >= 0.65 and i != disease_index:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        sub_auroc.append(auroc_val)
        sub_recall.append(recall_val)
        cnt += 1

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

left_hilar(1), auroc: 0.6876675102526184, recall: 0.4636363636363636
right_hilar(2), auroc: 0.6710241423677065, recall: 0.34408602150537637
hilar_unspec(3), auroc: 0.6987572633104338, recall: 0.5716612377850163
left_pleural(4), auroc: 0.8729606550803045, recall: 0.7701421800947867
right_pleural(5), auroc: 0.8817677433937102, recall: 0.7875417130144605
pleural_unspec(6), auroc: 0.8476208584704125, recall: 0.7807441067878443
heart_size(7), auroc: 0.7393268846743594, recall: 0.6426146010186757
left_diaphragm(9), auroc: 0.8178686254507821, recall: 0.6771084337349398
right_diaphragm(10), auroc: 0.8504442898175479, recall: 0.7305389221556886
diaphragm_unspec(11), auroc: 0.7190377630320863, recall: 0.6190476190476191
retrocardiac(12), auroc: 0.7643984596816951, recall: 0.6935123042505593
lower_left_lobe(13), auroc: 0.7357518177863898, recall: 0.6423357664233577
lower_right_lobe(15), auroc: 0.7102500101209619, recall: 0.614010989010989
middle_right_lobe(16), auroc: 0.687772374450575, recall: 0

(0.7598260715189474, 0.6318501512401405)

<IPython.core.display.Javascript object>

In [18]:
mask_select = np.array(all_auroc) >= 0.65
mask_select[disease_index] = False

torch_mask = torch.from_numpy(mask_select)
torch_mask.sum(), torch_mask.size()

(tensor(90), torch.Size([107]))

<IPython.core.display.Javascript object>

In [19]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
bad_auroc = []
bad_recall = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    if auroc_val <= 0.7:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        cnt += 1
        bad_auroc.append(auroc_val)
        bad_recall.append(recall_val)

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

trachea(0), auroc: 0.6430099772200941, recall: 0.43137254901960786
left_hilar(1), auroc: 0.6876675102526184, recall: 0.4636363636363636
right_hilar(2), auroc: 0.6710241423677065, recall: 0.34408602150537637
hilar_unspec(3), auroc: 0.6987572633104338, recall: 0.5716612377850163
heart_border(8), auroc: 0.5720200527399907, recall: 0.46632124352331605
upper_left_lobe(14), auroc: 0.6443296206527405, recall: 0.42328042328042326
middle_right_lobe(16), auroc: 0.687772374450575, recall: 0.36470588235294116
left_lower_lung(18), auroc: 0.6756972575589951, recall: 0.5403726708074534
left_mid_lung(19), auroc: 0.615527148058575, recall: 0.2182741116751269
left_upper_lung(20), auroc: 0.6419372467426528, recall: 0.3939393939393939
right_mid_lung(24), auroc: 0.6275597980898123, recall: 0.485
right_upper_lung(25), auroc: 0.6621302068970161, recall: 0.3422818791946309
lung_apices(28), auroc: 0.6911907687209451, recall: 0.5158730158730159
cardiophrenic_sulcus(33), auroc: 0.6767029025616047, recall: 0.4255

(0.7598260715189474, 0.6318501512401405)

<IPython.core.display.Javascript object>

In [20]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
auroc = []
recall = []
for i in range(len(concepts)):
    try:
        auroc.append(metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i]))
        recall.append(metrics.recall_score(data_GT[:, i], y_pred[:, i]))
    except ValueError:
        pass

print(len(auroc))

mean_auroc = np.mean(np.array(auroc))
mean_recall = np.mean(np.array(recall))
mean_auroc, mean_recall

107


(0.7391129587184638, 0.5993157936271764)

<IPython.core.display.Javascript object>

In [21]:
mean_auroc = np.mean(np.array(auroc))
mean_auroc

0.7391129587184638

<IPython.core.display.Javascript object>

In [22]:
cnt = 0
th = 0.65
concept_select = []

test_select_attr = []
test_select_proba_concept = []
test_select_logits_concept = []


val_select_attr = []
val_select_proba_concept = []
val_select_logits_concept = []


train_select_attr = []
train_select_proba_concept = []
train_select_logits_concept = []


test_select_proba_concept = []
for i in range(len(concepts)):
    if auroc[i] >= th and i != disease_index:
        test_select_attr.append(test_attr[:, i])
        test_select_proba_concept.append(test_proba_concept[:, i])
        test_select_logits_concept.append(test_logits_concept[:, i])

        val_select_attr.append(val_attr[:, i])
        val_select_proba_concept.append(val_proba_concept[:, i])
        val_select_logits_concept.append(val_logits_concept[:, i])

        train_select_attr.append(train_attr[:, i])
        train_select_proba_concept.append(train_proba_concept[:, i])
        train_select_logits_concept.append(train_logits_concept[:, i])

        concept_select.append(concepts[i])
        cnt += 1

test_select_attr = torch.stack(test_select_attr, dim=1)
test_select_proba_concept = torch.stack(test_select_proba_concept, dim=1)
test_select_logits_concept = torch.stack(test_select_logits_concept, dim=1)
val_select_attr = torch.stack(val_select_attr, dim=1)
val_select_proba_concept = torch.stack(val_select_proba_concept, dim=1)
val_select_logits_concept = torch.stack(val_select_logits_concept, dim=1)
train_select_attr = torch.stack(train_select_attr, dim=1)
train_select_proba_concept = torch.stack(train_select_proba_concept, dim=1)
train_select_logits_concept = torch.stack(train_select_logits_concept, dim=1)


print("Test sizes: ")
print(test_select_attr.size())
print(test_select_proba_concept.size())
print(test_select_logits_concept.size())
print("Val sizes: ")
print(val_select_attr.size())
print(val_select_proba_concept.size())
print(val_select_logits_concept.size())
print("Train sizes: ")
print(train_select_attr.size())
print(train_select_proba_concept.size())
print(train_select_logits_concept.size())
print(cnt)

Test sizes: 
torch.Size([24072, 90])
torch.Size([24072, 90])
torch.Size([24072, 90])
Val sizes: 
torch.Size([23449, 90])
torch.Size([23449, 90])
torch.Size([23449, 90])
Train sizes: 
torch.Size([187548, 90])
torch.Size([187548, 90])
torch.Size([187548, 90])
90


<IPython.core.display.Javascript object>

In [27]:
metric = "auroc"

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"torch_concepts_mask_{metric}.pt"),
    tensor_to_save=torch_mask,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_{metric}_attributes.pt"),
    tensor_to_save=test_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_proba_{metric}_concepts.pt"),
    tensor_to_save=test_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_logits_{metric}_concepts.pt"),
    tensor_to_save=test_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_{metric}_attributes.pt"),
    tensor_to_save=val_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_proba_{metric}_concepts.pt"),
    tensor_to_save=val_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_logits_{metric}_concepts.pt"),
    tensor_to_save=val_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_{metric}_attributes.pt"),
    tensor_to_save=train_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_proba_{metric}_concepts.pt"),
    tensor_to_save=train_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_logits_{metric}_concepts.pt"),
    tensor_to_save=train_select_logits_concept,
)

pickle.dump(
    concept_select,
    open(os.path.join(path, "dataset_g", f"selected_concepts_{metric}.pkl"), "wb"),
)

print(os.path.join(path, "dataset_g"))

/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/effusion/dataset_g


<IPython.core.display.Javascript object>

In [24]:
concept_select

['left_hilar',
 'right_hilar',
 'hilar_unspec',
 'left_pleural',
 'right_pleural',
 'pleural_unspec',
 'heart_size',
 'left_diaphragm',
 'right_diaphragm',
 'diaphragm_unspec',
 'retrocardiac',
 'lower_left_lobe',
 'lower_right_lobe',
 'middle_right_lobe',
 'upper_right_lobe',
 'left_lower_lung',
 'left_apical_lung',
 'left_lung_unspec',
 'right_lower_lung',
 'right_upper_lung',
 'right_apical_lung',
 'right_lung_unspec',
 'lung_apices',
 'lung_bases',
 'left_costophrenic',
 'right_costophrenic',
 'costophrenic_unspec',
 'cardiophrenic_sulcus',
 'mediastinal',
 'spine',
 'right_atrium',
 'right_ventricle',
 'aorta',
 'svc',
 'interstitium',
 'parenchymal',
 'cavoatrial_junction',
 'stomach',
 'opacity',
 'edema',
 'atelectasis',
 'tube',
 'consolidation',
 'process',
 'abnormality',
 'enlarge',
 'tip',
 'low',
 'line',
 'congestion',
 'catheter',
 'cardiomegaly',
 'air',
 'tortuous',
 'lead',
 'disease',
 'calcification',
 'device',
 'engorgement',
 'picc',
 'clip',
 'elevation',
 'nod

<IPython.core.display.Javascript object>

In [25]:
len(concept_select)

90

<IPython.core.display.Javascript object>

In [26]:
y_gt_tr = torch.load(os.path.join(path, "dataset_g", f"train_class_labels.pt"))
(y_gt_tr == 1).sum(dim=0), (y_gt_tr == 0).sum(dim=0)

(tensor([43998]), tensor([143550]))

<IPython.core.display.Javascript object>

In [28]:
((y_gt_tr == 1).sum(dim=0) / y_gt_tr.size(0)).item()

0.23459593951702118

<IPython.core.display.Javascript object>